# Plot ROI Contours and Corresponding Whole-Session Activity Traces

### Plots the mean motion-corrected projection image with select ROI contours displayed. Also plots each ROI's corresponding neuropil-corrected dF/F activity trace across the whole session.

Requires suite2p to have run through the loaded dataset.

### Parameters

__fdir__ : string 

    Root file directory containing the suite2p folder. IMPORTANT Note: leave off the last backslash, and include the letter r in front of string (treats the contents as a raw string). For example: r'C:\Users\my_user\analyze_sessions'

__tseries_start_end__ : list with two entries or None

    If set to `None`, the whole session's duration will be plotted for the time-series
    If set to a list with two ints/floats (eg. `[a,b]`), the time-series time window will be from time a to time b. For example, if `tseries_start_end` was set to `[0, 10]`, the time-series plot x limits will be 0 to 10 seconds.

__rois_to_plot__ can be one of the following: 
1) A list of select rois \
2) An integer (n) indicating n first rois to plot \
3) None which plots all valid ROIs 

In [ ]:
import os
import numpy as np
import h5py
import tifffile as tiff

import utils

import matplotlib.pyplot as plt
plt.rcParams['text.usetex'] = False
plt.rcParams['text.latex.unicode'] = False

In [ ]:
"""
USER-DEFINED VARIABLES
"""

fdir = os.path.abspath('./sample_data') # directory where suite2p folder is

tseries_start_end = [0, 40] # time window in seconds; can also set to None for whole session

"""
define number of ROIs to visualize

can be: 
1) a list of select rois, 
2) an integer (n) indicating n first rois to plot, or 
3) None which plots all valid ROIs
""" 
rois_to_plot = 5 

In [ ]:
def calc_dff(data):
    baseline = np.nanmean(data)
    return ( (data-baseline)/baseline )*100

In [ ]:
suite2p_dat_dir = os.path.join('suite2p','plane0')
fig_save_dir = os.path.join(fdir, 'figs')
utils.check_exist_dir(fig_save_dir)

s2p_data = {}
# define paths for loading s2p data
s2p_dir = os.path.join(fdir, 'suite2p', 'plane0')
s2p_F_path = os.path.join(s2p_dir, 'F.npy')
s2p_Fneu_path = os.path.join(s2p_dir, 'Fneu.npy')
s2p_iscell_path = os.path.join(s2p_dir, 'iscell.npy')
s2p_ops_path = os.path.join(s2p_dir, 'ops.npy')
s2p_stat_path = os.path.join(s2p_dir, 'stat.npy')

# load s2p data
s2p_data['F'] = np.load(s2p_F_path, allow_pickle=True)
s2p_data['Fneu'] = np.load(s2p_Fneu_path, allow_pickle=True)
s2p_data['iscell'] = np.load(s2p_iscell_path, allow_pickle=True)
s2p_data['ops'] = np.load(s2p_ops_path, allow_pickle=True).item()
s2p_data['stat'] = np.load(s2p_stat_path, allow_pickle=True)

s2p_data['F_npil_corr'] = s2p_data['F'] - s2p_data['ops']['neucoeff'] * s2p_data['Fneu']

s2p_data['F_npil_corr_dff'] = np.apply_along_axis(calc_dff, 1, s2p_data['F_npil_corr'])

In [ ]:
iscell_ids = np.where( s2p_data['iscell'][:,0] == 1 )[0] # indices of user-curated cells referencing all ROIs detected by s2p

if isinstance(rois_to_plot, int): # if int is supplied, first n user-curated rois included in analysis
    rois_to_plot = np.arange(rois_to_plot)
elif rois_to_plot is None: # if None is supplied, all user-curated rois included in analysis
    rois_to_plot = np.arange(len(iscell_ids))
    
cell_ids = iscell_ids[rois_to_plot] # indices of detected cells across all ROIs from suite2p
num_rois = len(rois_to_plot)

In [ ]:
# initialize templates for contour map
colors_roi = plt.cm.viridis(np.linspace(0,1,num_rois))
s2p_masks = np.empty([num_rois, s2p_data['ops']['Ly'], s2p_data['ops']['Lx']])
roi_centroids = np.empty([num_rois, 2])

# loop through ROIs and add their spatial footprints to template
for idx, roi_id in enumerate(cell_ids):
    
    zero_template = np.zeros([s2p_data['ops']['Ly'], s2p_data['ops']['Lx']])
    zero_template[ s2p_data['stat'][roi_id]['ypix'], s2p_data['stat'][roi_id]['xpix'] ] = 1
    s2p_masks[idx,...] = zero_template
    
    roi_centroids[idx,...] = [np.min(s2p_data['stat'][roi_id]['ypix']), np.min(s2p_data['stat'][roi_id]['xpix'])]
    
    if idx == num_rois-1:
        break


In [ ]:
to_plot = s2p_data['ops']['meanImg']

fig, ax = plt.subplots(1, 1, figsize = (10,10))
ax.imshow(to_plot, cmap = 'gray', vmin=np.min(to_plot)*1.0, vmax=np.max(to_plot)*0.5)
ax.axis('off')

for idx, roi_id in enumerate(cell_ids): 
    ax.contour(s2p_masks[idx,:,:], colors=[colors_roi[idx]])
    ax.text(roi_centroids[idx][1]-1, roi_centroids[idx][0]-1,  str(idx), fontsize=15, color = colors_roi[idx]);

plt.savefig(os.path.join(fig_save_dir, 'roi_contour_map.png'))
plt.savefig(os.path.join(fig_save_dir, 'roi_contour_map.pdf'))

# Plot Time-series of Selected ROIs

In [ ]:
# initialize variables for plotting time-series
fs = s2p_data['ops']['fs']
num_samps = s2p_data['ops']['nframes']
total_time = num_samps/fs 
tvec = np.linspace(0,total_time,num_samps)

In [ ]:
# F_npil_corr_dff contains all s2p-detected cells; cell_ids references those indices
trace_data_selected = s2p_data['F_npil_corr_dff'][cell_ids]


fig, ax = plt.subplots(num_rois, 1, figsize = (9,2*num_rois))
for idx in range(num_rois):
    
    to_plot = trace_data_selected[idx] 
    
    ax[idx].plot(tvec, np.transpose( to_plot ), color = colors_roi[idx] );
    
    if idx == np.ceil(num_rois/2-1):
        ax[idx].set_ylabel('Fluorescence Level',fontsize = 20)
        
# Setting the values for all axes.
if tseries_start_end is None:
    xlims = [0,tvec[-1]]
else:
    xlims = tseries_start_end
plt.setp(ax, xlim=xlims, ylim=[np.min(trace_data_selected)+np.min(trace_data_selected)*0.1, 
                                      np.max(trace_data_selected)+np.max(trace_data_selected)*0.1])  

ax[idx].set_xlabel('Time (s)',fontsize = 20);

plt.savefig(os.path.join(fig_save_dir, 'roi_ts.png'))
plt.savefig(os.path.join(fig_save_dir, 'roi_ts.pdf'))